In [1]:
import pandas as pd
import json

# Assuming we have the following data
data = {
    'transaction_id': ['T1001', 'T1002', 'T1003', 'T1004', 'T1005'],
    'customer_id': ['C001', 'C002', 'C003', 'C002', 'C001'],
    'payment_amount': [125.50, 89.99, 120.00, 54.30, 210.20],
    'payment_date': ['2021-10-05', '2021-10-06', '2021-10-07', '2021-10-05', '2021-10-08'],
    'payment_status': ['Paid', 'Unpaid', 'Paid', 'Paid', 'Pending']
}

# Create DataFrame
df = pd.DataFrame(data)

In [2]:
def retrieve_payment_status(df, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values: 
        return json.dumps({'status': df[df.transaction_id == transaction_id].payment_status.item()})
    return json.dumps({'error': 'transaction id not found.'})

def retrieve_payment_date(df, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values: 
        return json.dumps({'date': df[df.transaction_id == transaction_id].payment_date.item()})
    return json.dumps({'error': 'transaction id not found.'})

In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_status",
            "description": "Get payment status of a transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction id.",
                    }
                },
                "required": ["transaction_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_date",
            "description": "Get payment date of a transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction id.",
                    }
                },
                "required": ["transaction_id"],
            },
        },
    }
]

In [4]:
import functools

names_to_functions = {
    'retrieve_payment_status': functools.partial(retrieve_payment_status, df=df),
    'retrieve_payment_date': functools.partial(retrieve_payment_date, df=df)
}

In [43]:
messages = [{"role": "user", "content": "Какая дата у транзакции T1001"}]

In [44]:
import os
from mistralai import Mistral

api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-large-latest"

client = Mistral(api_key=api_key)
response = client.chat.complete(
    model = model,
    messages = messages,
    tools = tools,
    tool_choice = "any",
)
response

ChatCompletionResponse(id='4b417b0ccc5f4e149a7a0841599e3f05', object='chat.completion', model='mistral-large-latest', usage=UsageInfo(prompt_tokens=167, completion_tokens=30, total_tokens=197), created=1739730490, choices=[ChatCompletionChoice(index=0, message=AssistantMessage(content='', tool_calls=[ToolCall(function=FunctionCall(name='retrieve_payment_date', arguments='{"transaction_id": "T1001"}'), id='K1XQS1EXO', type=None, index=0)], prefix=False, role='assistant'), finish_reason='tool_calls')])

In [45]:
messages.append(response.choices[0].message)

In [46]:
import json

tool_call = response.choices[0].message.tool_calls[0]
function_name = tool_call.function.name
function_params = json.loads(tool_call.function.arguments)
print("\nfunction_name: ", function_name, "\nfunction_params: ", function_params)


function_name:  retrieve_payment_date 
function_params:  {'transaction_id': 'T1001'}


In [47]:
function_result = names_to_functions[function_name](**function_params)
function_result

'{"date": "2021-10-05"}'

In [48]:
messages.append({"role":"tool", "name":function_name, "content":function_result, "tool_call_id":tool_call.id})

response = client.chat.complete(
    model = model, 
    messages = messages
)
response.choices[0].message.content

'Дата транзакции T1001 — 5 октября 2021 года.'

In [12]:
df

,transaction_id,customer_id,payment_amount,payment_date,payment_status
0,T1001,C001,125.50,2021-10-05,Paid
1,T1002,C002,89.99,2021-10-06,Unpaid
2,T1003,C003,120.00,2021-10-07,Paid
3,T1004,C002,54.30,2021-10-05,Paid
4,T1005,C001,210.20,2021-10-08,Pending


In [49]:
messages

[{'role': 'user', 'content': 'Какая дата у транзакции T1001'},
 AssistantMessage(content='', tool_calls=[ToolCall(function=FunctionCall(name='retrieve_payment_date', arguments='{"transaction_id": "T1001"}'), id='K1XQS1EXO', type=None, index=0)], prefix=False, role='assistant'),
 {'role': 'tool',
  'name': 'retrieve_payment_date',
  'content': '{"date": "2021-10-05"}',
  'tool_call_id': 'K1XQS1EXO'}]